In [1]:
pip install XGBoost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import precision_recall_cutoff
import Ensemble
from xgboost import XGBClassifier
from cost_function import cost_function, cost_function_cutoff
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


# Reading the csv file
train = pd.read_csv('train_dataset.csv')

# Defining the file to be read from s3 bucket
test = pd.read_csv('test_dataset.csv')
test = test.dropna()
test.shape

(498121, 44)

In [3]:
train.shape

(1879, 45)

In [4]:
# Defining input and target variables
test = test[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2', 'interaction_5']]
X = train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2', 'interaction_5']]
Y = train['fraud']

# Splitting the data into train, test, and validation
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2)


# Scaling the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)

X_val = scaler.fit_transform(X_val)
X_val = pd.DataFrame(X_val, columns = X.columns)


X_val

,totalScanTimeInSeconds,interaction_1,interaction_4,heredity_1,interaction_9,heredity_2,interaction_5
0,0.472070,0.0,0.0,0.0,0.0,0.0,0.818182
1,0.104600,1.0,0.0,1.0,1.0,0.2,0.254545
2,0.848302,0.0,1.0,0.0,0.0,0.0,0.000000
3,0.317087,0.0,1.0,0.0,0.0,0.0,0.654545
4,0.066265,0.0,0.0,0.0,0.0,0.0,0.272727
...,...,...,...,...,...,...,...
371,0.026287,0.0,1.0,0.0,0.0,0.0,0.436364
372,0.640197,0.0,0.0,0.0,0.0,0.0,0.000000
373,0.833516,0.0,0.0,0.0,0.0,0.0,0.000000
374,0.834611,0.0,1.0,0.0,0.0,0.0,0.045455


In [5]:
# Defining top 7, 6, and 5 variables
# Train dataset
X_train_7 = X_train
X_train_6 = X_train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_train_5 = X_train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]

# Validation dataset
X_val_7 = X_val
X_val_6 = X_val[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_val_5 = X_val[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]

# Validation dataset
test_7 = test
test_6 = test[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
test_5 = test[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]


### The Best Random Forest Model

In [6]:
RF_md = RandomForestClassifier(max_depth=3, min_samples_leaf=7, min_samples_split=10, n_estimators=500).fit(X_train_5, Y_train)

# Predicting on validation and test
RF_val_pred = RF_md.predict_proba(X_val_5)[:,1]
RF_test_pred = RF_md.predict_proba(test_5)[:,1]

RF_val_pred

array([7.30049092e-03, 7.51752838e-02, 4.07303998e-03, 1.02178178e-04,
       7.44658499e-03, 4.07303998e-03, 1.02178178e-04, 3.94028868e-01,
       1.02178178e-04, 3.11270421e-04, 4.59078862e-03, 3.11270421e-04,
       1.02178178e-04, 4.59078862e-03, 7.23749275e-03, 4.42842437e-03,
       8.58838060e-02, 3.11270421e-04, 3.56606895e-03, 7.44658499e-03,
       7.30049092e-03, 7.44658499e-03, 1.02178178e-04, 1.09337627e-01,
       7.30049092e-03, 1.02178178e-04, 3.11270421e-04, 7.44658499e-03,
       1.36146331e-02, 1.02178178e-04, 4.42842437e-03, 3.56606895e-03,
       7.44658499e-03, 7.30049092e-03, 5.02756483e-03, 7.37103467e-02,
       3.11270421e-04, 1.02178178e-04, 1.02178178e-04, 7.44658499e-03,
       4.07303998e-03, 3.11270421e-04, 3.11270421e-04, 3.11270421e-04,
       4.59078862e-03, 7.44012912e-02, 1.02178178e-04, 4.42842437e-03,
       4.42842437e-03, 3.11270421e-04, 1.38865778e-02, 1.02178178e-04,
       7.44303559e-03, 6.13205543e-02, 3.11270421e-04, 7.44658499e-03,
      

### The Best Support Vector Machine Model

In [7]:
SVC_md = SVC(C=0.01, gamma=0.1, kernel='sigmoid', probability=True).fit(X_train_5, Y_train)

# Predicting on validation and test
SVC_val_pred = SVC_md.predict_proba(X_val_5)[:,1]
SVC_test_pred = SVC_md.predict_proba(test_5)[:,1]

SVC_test_pred

array([2.10304326e-05, 2.44834874e-05, 1.80981201e-06, ...,
       1.13763524e-05, 2.45035584e-05, 2.45034381e-05])

### The Best AdaBoost Model

In [8]:
ADA_md = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         min_samples_leaf=5,
                                                         min_samples_split=10),
                                                         learning_rate=0.01, n_estimators=300).fit(X_train_7, Y_train)
# Predicting on validation and test
ADA_val_pred = ADA_md.predict_proba(X_val_7)[:,1]
ADA_test_pred = ADA_md.predict_proba(test_7)[:,1]

ADA_test_pred

array([0.01141731, 0.00425823, 0.31357652, ..., 0.00425823, 0.32924427,
       0.0136434 ])

### The Best XGBClassifier Model

In [ ]:
XGBoost_md = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0.3, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=5, min_child_weight=5,
              monotone_constraints='()', n_estimators=500, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None).fit(X_train_5, Y_train)

# Predicting on validation and test
XGBoost_val_pred = XGBoost_md.predict_proba(X_val_7)[:,1]
XGBoost_test_pred = XGBoost_md.predict_proba(test_7)[:,1]

XGBoost_val_pred

In [ ]:
esemble_train = pd.DataFrame({'RF': RF_val_pred, "SVC": SVC_val_pred, 'ADA': ADA_val_pred, "XGBoost": XGBoost_val_pred, "Y_val": Y_val})
esemble_train.head()

In [ ]:
esemble_test = pd.DataFrame({"RF_preds_test": RF_test_pred, "SVC_preds_test": SVC_test_pred, "ADA_preds_test": ADA_test_pred, "XGBoost_preds_test": XGBoost_test_pred})
esemble_test.head()

In [ ]:
import Ensemble
RF_val_pred = pd.DataFrame(RF_val_pred)
SVC_val_pred = pd.DataFrame(SVC_val_pred)
ADA_val_pred = pd.DataFrame(ADA_val_pred)
XGBoost_val_pred = pd.DataFrame(XGBoost_val_pred)
Y = esemble_train['Y_val']
RF_test_pred = pd.DataFrame(RF_test_pred)
SVC_test_pred = pd.DataFrame(SVC_test_pred)
ADA_test_pred = pd.DataFrame(ADA_test_pred)
XGBoost_test_pred = pd.DataFrame(XGBoost_test_pred)

ensemble = Ensemble.ensemble(RF_val_pred, SVC_val_pred, ADA_val_pred, XGBoost_val_pred, Y, RF_test_pred, SVC_test_pred, ADA_test_pred, XGBoost_test_pred)


In [ ]:
plt.hist(ensemble)
plt.show()

In [ ]:
ensemble = pd.DataFrame(ensemble)

In [ ]:
Predictions = pd.concat([RF_test_pred, SVC_test_pred, ADA_test_pred, ensemble], axis = 1)
Predictions.columns = ['RF_test_pred', 'SVC_test_pred', 'ADA_test_pred', 'Ensemble']
Predictions

In [ ]:
likelyhoods = pd.DataFrame({'Likelyhoods': Predictions['Ensemble']})
likelyhoods.head()

In [ ]:
likelyhoods.to_csv("likelyhoods.csv", index = False)

# Indentifyung the optimal cut-off value
opt_cutoff = cost_function_cutoff(Y_val, SVC_val_pred)

# Changing the likelihoods to labels
SVC_label = np.where(SVC_test_pred < opt_cutoff, 0, 1)

# Computing the confusion matrix
X = confusion_matrix(Y_test, SVC_label)
print(X)
print('The cost of the SVC model is: ', -1500*X[1,0] - 1000*X[0,1] + 500*X[1, 1])